In [ ]:
import sys
sys.path.append("..")

import random

import time
import math
import random
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Optional, Union

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cluster import KMeans

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd

from src.datasets import *
from src.util import *
from src.util.image import * 
from src.algo import Space2d, IFS
from src.datasets import *
from src.models.cnn import *
from src.util.embedding import *
from src.models.clip import ClipSingleton

In [ ]:
def plot_samples(
        iterable, 
        total: int = 32, 
        nrow: int = 8, 
        return_image: bool = False, 
        show_compression_ratio: bool = False,
        label: Optional[Callable] = None,
):
    samples = []
    labels = []
    f = ImageFilter()
    try:
        for idx, entry in enumerate(tqdm(iterable, total=total)):
            image = entry
            if isinstance(entry, (list, tuple)):
                image = entry[0]
            if image.ndim == 4:
                image = image.squeeze(0)
            samples.append(image)
            if show_compression_ratio:
                labels.append(round(f.calc_compression_ratio(image), 3))
            elif label is not None:
                labels.append(label(entry) if callable(label) else idx)
                
            if len(samples) >= total:
                break
    except KeyboardInterrupt:
        pass
    
    if labels:
        image = VF.to_pil_image(make_grid_labeled(samples, nrow=nrow, labels=labels))
    else:
        image = VF.to_pil_image(make_grid(samples, nrow=nrow, pad_value=0))
    if return_image:
        return image
    display(image)

In [ ]:
ds = RpgTileIterableDataset(shape=(3, 32, 32))

In [ ]:
plot_samples(ds)

In [ ]:
ds_train = SplitIterableDataset(ds, ratio=20, train=True)
ds_test  = SplitIterableDataset(ds, ratio=20, train=False)
print(len(list(ds_train)))
print(len(list(ds_test)))

In [ ]:
images = next(iter(DataLoader(ds, batch_size=1_000_000)))

In [ ]:
images.shape

In [ ]:
torch.save((images.clamp(0, 1) * 255).to(torch.uint8), "../datasets/rpg-3x32x32-uint.pt")

# PCA

In [ ]:
pca = PCA(128)
pca.fit(images.flatten(1))

In [ ]:
VF.to_pil_image(make_grid(torch.Tensor(pca.components_).reshape(-1, *ds.shape), normalize=True))

In [ ]:
features = torch.Tensor(pca.transform(images.flatten(1)))

In [ ]:
clusterer = KMeans(100, n_init="auto")
labels = clusterer.fit_predict(features)

In [ ]:
hist, _ = np.histogram(labels, clusterer.n_clusters, (0, clusterer.n_clusters))
hist.sort()
px.bar(hist)

In [ ]:
grid_images = []
grid_labels = []
for label in range(clusterer.n_clusters):
    indices = np.argwhere(labels == label).reshape(-1)[:30]
    sample = images[indices]
    for s in sample:
        grid_images.append(s)
    for i in indices:
        grid_labels.append(i)
        
display(VF.to_pil_image(make_grid(
    grid_images, nrow=32, 
    #labels=grid_labels
)))

In [ ]:
ana = FactorAnalysis(128)
features2 = ana.fit_transform(images.flatten(1))

In [ ]:
VF.to_pil_image(make_grid(torch.Tensor(ana.components_).reshape(-1, *ds.shape), normalize=True))

In [ ]:
clusterer = KMeans(100, n_init="auto")
labels2 = clusterer.fit_predict(features2)

hist, _ = np.histogram(labels, clusterer.n_clusters, (0, clusterer.n_clusters))
hist.sort()
px.bar(hist)

In [ ]:
grid_images = []
grid_labels = []
for label in range(clusterer.n_clusters):
    indices = np.argwhere(labels2 == label).reshape(-1)[:30]
    sample = images[indices]
    for s in sample:
        grid_images.append(s)
    for i in indices:
        grid_labels.append(i)
        
display(VF.to_pil_image(make_grid(
    grid_images, nrow=32, 
    #labels=grid_labels
)))

# via AutoEncoder

In [ ]:
from scripts.train_autoencoder_vae import VariationalAutoencoderConv
model = VariationalAutoencoderConv((3, 32, 32), channels=[16, 24, 32], kernel_size=5, latent_dims=128)
model.load_state_dict(torch.load("../checkpoints/vae-rpg-1/snapshot.pt")["state_dict"])

In [ ]:
@torch.no_grad()
def model_forward(model: nn.Module, x: torch.Tensor, batch_size: int = 128) -> torch.Tensor:
    outputs = []
    with tqdm(desc=type(model).__name__, total=x.shape[0]) as progress:
        for i in range(0, x.shape[0], batch_size):
            outputs.append(model(x[i: i + batch_size]))
            progress.update(outputs[-1].shape[0])
    return torch.concat(outputs, dim=0)
        

In [ ]:
with torch.no_grad():
    features3 = normalize_embedding(model_forward(model.encoder, images))

In [ ]:
torch.save(features3, "tmp_features3.pt")

In [ ]:
features3 = torch.load("tmp_features3.pt")

In [ ]:
clusterer = KMeans(100, n_init="auto")
labels3 = clusterer.fit_predict(features3)

hist, _ = np.histogram(labels3, clusterer.n_clusters, (0, clusterer.n_clusters))
hist.sort()
px.bar(hist)

In [ ]:
grid_images = []
grid_labels = []
for label in range(clusterer.n_clusters):
    indices = np.argwhere(labels3 == label).reshape(-1)[:30]
    sample = images[indices]
    for s in sample:
        grid_images.append(s)
    for i in indices:
        grid_labels.append(i)
        
display(VF.to_pil_image(make_grid(
    grid_images, nrow=32, 
    #labels=grid_labels
)))

In [ ]:
with torch.no_grad():
    sim3 = features3 @ features3.T
    #sim_indices = sim3.argsort(1)
    #print(sim_indices.shape)

In [ ]:
px.imshow(sim3[:100, :100], height=700)

In [ ]:
with torch.no_grad():
    sorted_indices = sim3[:1000].argsort(dim=-1)
sorted_indices.shape

In [ ]:
VF.to_pil_image(make_grid(images[sorted_indices[:10, :10].flatten(0)], nrow=10))

In [ ]:
sorted_indices[:10, :10]